In [8]:
%load_ext autoreload
%autoreload 2

import random
import numpy as np
from jassbot.Config import *
from jassbot.GameLogic import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
state = init()
new_game(state)
print_state(state)
while len(state['played']) < 36:
    play_round(state)
print_state(state)

init game
new game
---
trump: spade
current: []
player 0: ['spade_j', 'club_a', 'heart_7', 'club_10', 'heart_6', 'spade_8', 'diamond_7', 'heart_j', 'heart_8']
-------------------------------
player 1: ['spade_k', 'club_6', 'heart_10', 'spade_10', 'spade_7', 'diamond_10', 'heart_9', 'spade_6', 'spade_a']
player 2: ['diamond_9', 'club_7', 'heart_q', 'club_8', 'club_j', 'diamond_a', 'heart_k', 'diamond_6', 'diamond_q']
player 3: ['spade_q', 'diamond_j', 'heart_a', 'club_9', 'diamond_k', 'club_k', 'diamond_8', 'spade_9', 'club_q']
team 0: 0 pts, team 1: 0 pts
played: []
-------------------------------
player 0 plays spade_j
player 1 plays spade_10
player 2 plays club_j
player 3 plays spade_q
player 0 & team 0 wins the round: 35 points
player 0 plays heart_6
player 1 plays heart_10
player 2 plays heart_q
player 3 plays heart_a
player 3 & team 1 wins the round: 24 points
player 3 plays club_q
player 0 plays club_a
player 1 plays spade_k
player 2 plays club_8
player 1 & team 1 wins the round: